## Combining IoT Botnet Dataset

We are using Bot-IoT dataset created by Koroniotis *et al*. The information about the dataset is given at https://ieee-dataport.org/documents/bot-iot-dataset

As mentioned in the description, the entire dataset contains more than 72.000.000 records. The authors also created a small-scale version of the dataset capturing 5% of the entire dataset carrying about 3 million of records. The small-scale version is stored into 4 csv files which can be downloaded [here](https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE?path=%2FCSV%2FTraning%20and%20Testing%20Tets%20(5%25%20of%20the%20entier%20dataset)%2FAll%20features).

In this program, we simply combine all 4 csv files into a single csv for convenient access. In the process, we also remove several columns which are either redundant or non-essential for our classification. 

The output csv file is called `MergedData.csv`.

In our paper, we use `MergedData2.csv` which is the combination of 3 csv files include the 1st, 2nd and 4th csv files.

In [1]:
import pandas as pd

# specify the 4 csv input files
all_filenames = [ "UNSW_2018_IoT_Botnet_Full5pc_1.csv",
                  "UNSW_2018_IoT_Botnet_Full5pc_2.csv", 
                  #"UNSW_2018_IoT_Botnet_Full5pc_3.csv", # in our paper, we exclude this csv 
                  "UNSW_2018_IoT_Botnet_Full5pc_4.csv" ]

# combine all files
df = pd.concat([pd.read_csv(f) for f in all_filenames])
df # preview the csv dataframe

/tmp/ipykernel_123473/4069243050.py:10: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in all_filenames])
/tmp/ipykernel_123473/4069243050.py:10: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in all_filenames])
/tmp/ipykernel_123473/4069243050.py:10: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in all_filenames])


,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.127040,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.200000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.127040,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.127040,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.127040,96,75,1.135260,1.129970,770,602,1,DoS,HTTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
668518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
668519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
668520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


### Removing individual records

In the dataset, source port (sport) and destination port (dport) contain port numbers in integer, but there is a small amount of ICMP packets containing a number in hex representation in string format. This is also the reason causing `DtypeWarning` while loading the csv files. We shall remove these records from our dataset.

In [2]:
(num_records,_) = df.shape
print(f"Number of records loaded = {num_records}")

print(f"\nRemoving records where `sport` and `dport` are not in integer format:")
df = df[pd.to_numeric(df['sport'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['dport'], errors='coerce').notnull()]
df = df.astype({'sport':int,'dport':int})
num_records_removed = num_records - df.shape[0]
print(f"- {num_records_removed} records removed.")
print(f"- Number of valid records = {df.shape[0]}")

Number of records loaded = 2668522

Removing records where `sport` and `dport` are not in integer format:
- 9044 records removed.
- Number of valid records = 2659478


### Relabelling attack categories

Attack types are given in `category` and `subcategory` columns in the csv file. Particularly for `DoS` and `DDoS` categories, each has 3 subcategories, namely `HTTP`, `TCP` and `UDP`. For other categories, their subcategory is unique.

We shall combine both columns and apply unique labels to differentiate `DoS` and `DDoS`. Other subcategories remain unchanged:
- label `DoS` + `HTTP` as `DH`
- label `DoS` + `TCP`  as `DT`
- label `DoS` + `UDP`  as `DP`
- label `DDoS` + `HTTP` as `DDH`
- label `DDoS` + `TCP`  as `DDT`
- label `DDoS` + `UDP`  as `DDP`

In [3]:
df.loc[(df['category']=='DoS') & (df['subcategory']=='HTTP'), 'subcategory'] = 'DH'
df.loc[(df['category']=='DoS') & (df['subcategory']=='TCP'), 'subcategory'] = 'DT'
df.loc[(df['category']=='DoS') & (df['subcategory']=='UDP'), 'subcategory'] = 'DP'
df.loc[(df['category']=='DDoS') & (df['subcategory']=='HTTP'), 'subcategory'] = 'DDH'
df.loc[(df['category']=='DDoS') & (df['subcategory']=='TCP'), 'subcategory'] = 'DDT'
df.loc[(df['category']=='DDoS') & (df['subcategory']=='UDP'), 'subcategory'] = 'DDP'
df # preview the csv dataframe

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.127040,96,75,1.133720,1.129970,770,602,1,DoS,DH
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.200000,1,2,0.005142,0.005142,2,6,1,DoS,DH
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.127040,96,75,1.135100,1.129970,770,602,1,DoS,DH
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.127040,96,75,1.135140,1.129970,770,602,1,DoS,DH
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.127040,96,75,1.135260,1.129970,770,602,1,DoS,DH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
668518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
668519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
668520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


### Removing redundant and non-essential features

Redundant and non-essential features are specified in `columns_to_remove`, and they shall be removed.

In [4]:
columns_to_remove = ["pkSeqID", "saddr", "daddr", "flgs", "state", "proto", "attack", "category"]
print("Before:")
print(str(list(df.columns)) + ", count = %d"%len(list(df.columns)))
df.drop(columns_to_remove, axis=1, inplace=True)
print("After:")
print(str(list(df.columns)) + ", count = %d"%len(list(df.columns)))

Before:
['pkSeqID', 'stime', 'flgs', 'flgs_number', 'proto', 'proto_number', 'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state', 'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP', 'attack', 'category', 'subcategory'], count = 46
After:
['stime', 'flgs_number', 'proto_number', 'sport', 'dport', 'pkts', 'bytes', 'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN

### Attack statistics

In [5]:
df['subcategory'].value_counts(dropna=False)

DP                   1032961
DT                    615800
DDP                   576876
DDT                   348751
Service_Scan           64280
OS_Fingerprint         17780
DH                      1485
DDH                      989
Normal                   477
Keylogging                73
Data_Exfiltration          6
Name: subcategory, dtype: int64

### Writing the dataframe into a single csv file

In [6]:
df.to_csv("MergedData.csv", index=False, encoding='utf-8-sig')